<h1 align="center"><a href="https://github.com/sborquez/her2bdl"> Her2BDL</a> - Her2 Bayesian Deep Learning</h1>

<br>
<center>
<img src="images/utfsm.png" width="50%", align="center"/>
</center>
<h2 align="center">Train Model</h2>

<center>
<i> Notebook created by Sebastián Bórquez G. - <a href="mailto://sebstian.borquez.g@gmail.com">sebastian.borquez.g@gmail.com</a> - her2bdl - August 2020.</i>
</center>


# Setup Notebook

The first step is to sync this notebook with Google Drive, and change directory to gerumo repository.

In [ ]:
#!export HER2BDL_HOME="/home/sborquez/projects/her2bdl"
#!export HER2BDL_DATASETS="/home/sborquez/projects/her2bdl/datasets"
#!export HER2BDL_EXPERIMENTS="/home/sborquez/projects/her2bdl/experiments"
#!export HER2BDL_EXTRAS="/home/sborquez/projects/her2bdl-files"

In [ ]:
## (Option A) Colab Setup
# Connect to your `Google Drive` and install some dependencies.
try:
    raise NotImplementedError
    from google.colab import drive
    colab_ = True
    drive.mount('/content/drive')
    #!pip install git+https://github.com/sborquez/her2bdl\
## (Option B) Local Setup
# Run locally.
except:
    colab_ = False
    
# Adhoc modules
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_DISABLE_PTX_JIT'] = "1"
os.environ["WANDB_MODE"] = 'dryrun'

from extra import *

# Her2BDL package
#if is not installed run these two line
HER2BDL_HOME=os.environ.get("HER2BDL_HOME")
%cd $HER2BDL_HOME

from her2bdl import *
%matplotlib inline

In [ ]:
config_folder = Path(HER2BDL_HOME) / "scripts" / "config"

# Configuration

In [ ]:
EPISTEMIC=False

In [ ]:
# Binary
#config_file = config_folder / "binary_classification/efficientnet_b0_binary_classifier.yaml"
#config_file = config_folder / "aleatoric/efficientnet_b0_binary_classifier.yaml"

# HER2
#config_file = config_folder / "baseline/hdxconv_c876_d02.yaml"
#config_file = config_folder / "baseline/hdxconv_c876_d05.yaml"
#config_file = config_folder / "baseline/hdxconv_c876_d08.yaml"
config_file = config_folder / "aleatoric/hdxconv_c876.yaml"


# Model Training

Load and fit models.

In [ ]:
config = load_config_file(config_file)
run_dir = setup_experiment(config, mode="training")

# Experiment paths and indentifiers
experiments_folder = config["experiment"]["experiments_folder"]
experiment_name    = config["experiment"]["name"]
experiment_folder  = Path(experiments_folder) / experiment_name
run_id             = config["experiment"]["run_id"]

# Training parameters
epochs = config["training"]["epochs"]
batch_size  = config["training"]["batch_size"]
# Dataset
data_configuration = config["data"]
generators, input_shape, num_classes, labels = setup_generators(
    batch_size=batch_size, **data_configuration
)
train_, val_ = generators
(train_dataset, steps_per_epoch) = train_
(val_dataset, validation_steps)  = val_
# Model architecture
model_configuration = config["model"]

In [ ]:
## Optimizer
optimizer_name = config["training"]["optimizer"]["name"]
optimizer_learning_rate = float(config["training"]["optimizer"]["learning_rate"]) # fix scientific notation parsed as str.
optimizer_parameters = config["training"]["optimizer"].get("parameters", {})
optimizer_parameters = optimizer_parameters or {}
optimizer = OPTIMIZERS[optimizer_name](
    learning_rate=optimizer_learning_rate, 
    **optimizer_parameters
)
## Class Weights
class_weight = None
class_weight_list = config["training"].get("class_weight", None)
if class_weight_list is not None:
    class_weight = {i: w for i, w in enumerate(class_weight_list)}
## Callbacks
enable_wandb = config["training"]["callbacks"]["enable_wandb"]
earlystop = config["training"]["callbacks"]["earlystop"]
experiment_tracker = config["training"]["callbacks"]["experiment_tracker"]
checkpoints = config["training"]["callbacks"]["checkpoints"]

## Epistemic Model
if EPISTEMIC:
    print("Training Epistemic Model")
    model = setup_model(input_shape, num_classes, **model_configuration)
    loss_function    = config["training"]["loss"]["function"]
    loss_parameters  = config["training"]["loss"]["parameters"]
    loss_parameters  = loss_parameters or {}
    loss = LOSS[loss_function](**loss_parameters)
    uncertainty_type="epistemic"
## Aleatoric Model
else:
    print("Aleatoric Model")
    model_ = setup_model(input_shape, num_classes, **model_configuration, build=True)
    model = model_.get_aleatoric_model()

    ## Loss
    loss = model.build_aleatoric_loss()
    uncertainty_type = "aleatoric"

## Fit model

In [ ]:
# Set Callback
callbacks = setup_callbacks(
     validation_data=val_dataset, 
     validation_steps=validation_steps,
     model_name=experiment_name,
     batch_size=batch_size,
     enable_wandb=enable_wandb,
     labels=labels,
     earlystop=earlystop,
     experiment_tracker=experiment_tracker,
     checkpoints=checkpoints,
     run_dir=run_dir,
     uncertainty_type=uncertainty_type
)

# Train
model.compile(
    optimizer=optimizer,
    loss=loss
)
history = model.fit(train_dataset, 
    verbose = 2,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset, 
    validation_steps=validation_steps,
    epochs=epochs,
    class_weight=class_weight,
    callbacks=callbacks
)

## WandB Sync 

In [ ]:
wandb.finish()
os.environ["WANDB_MODE"] = ""

In [ ]:
reset_kernel()

### 